In [1]:
#https://github.com/amazon-science/efficient-longdoc-classification
from functools import partial
import nltk
import pickle as pk
import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_bart import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed_everything(1376)

Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [2]:
z=torch.ones((1,2),dtype=torch.long,device='cuda')
z.device

device(type='cuda', index=0)

In [13]:
configs

Namespace(is_baseline=False, model_base='facebook/bart-base', output_dir='trained_models_sum_boundary/', task_type='xsum', run_id='bart_base_model_context_enforcer_250', eval_steps=1000, learning_rate=5e-05, max_seq_len=800, evaluation_strategy='epoch', save_strategy='epoch', seed=10, lr_scheduler_type='linear', weight_decay=0.0, warmup_ratio=0.25, num_train_epochs=10, save_total_limit=1, per_device_train_batch_size=12, per_device_eval_batch_size=12, gradient_accumulation_steps=4, fp16=True, verbose=False, context_max_len=250)

In [2]:
configs= pk.load(open("trained_models_sum_boundary/bart_base_model_context_enforcer_250/train_args.ap",'rb'))
context_max_len=configs.context_max_len
context_max_len_list = [context_max_len]#,300,450]
context_sampling_bounds=(0.1, 0.45)

In [18]:
configs.task_type

'xsum'

In [3]:
tokenizer = create_text_tokenizer(configs.model_base)

task_dataset_gen = get_dataset_specified_tasks(configs.task_type)

train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs.model_base,
        context_max_len = context_max_len,
        context_sampling_bounds = context_sampling_bounds,
        context_max_len_list= context_max_len_list,#is_baseline=True
    )

Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
generator = model_builder()
train_model_path = "trained_models_sum_boundary/bart_base_model_context_enforcer_multi/checkpoint-34008/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Some weights of BartForContextualRecovery were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.layers.2.context_enforcer._left_mha.out_proj.weight', 'encoder.layers.1.context_enforcer._left_mha.in_proj_weight', 'encoder.layers.4.context_enforcer._right_mha.out_proj.bias', 'encoder.layers.0.context_enforcer._wlc.weight', 'encoder.layers.4.context_enforcer._left_mha.out_proj.weight', 'encoder.layers.3.context_enforcer._wlc.bias', 'encoder.layers.3.context_enforcer._wrc.bias', 'encoder.layers.3.context_enforcer_layer_norm.weight', 'encoder.layers.1.context_enforcer._left_mha.in_proj_bias', 'encoder.layers.5.context_enforcer._right_mha.out_proj.bias', 'encoder.layers.4.context_enforcer_layer_norm.bias', 'encoder.layers.1.context_enforcer._wrc.weight', 'encoder.layers.2.context_enforcer._wlc.bias', 'encoder.layers.3.context_enforcer._right_mha.out_proj.bias', 'encoder.layers.4.context_enforcer._left_mha.in_proj_weight', 'encoder.layers.0.

In [5]:
te1= test_dataset[6]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

Token indices sequence length is longer than the specified maximum sequence length for this model (1187 > 1024). Running this sequence through the model will result in indexing errors


In [6]:
seed_everything(1376)
boundary_sample =  (0.1, 0.45)
seq_len = te1.input_ids[:1024].shape[0]

boundary_start = int(0.45*seq_len) 
context_boundary = compute_context_boundary(seq_len,context_max_len=context_max_len)
context_boundary

Global seed set to 1376


(264, 514)

In [7]:
output = generator(input_ids=b_input_ids[:,:1000],
                attention_mask=b_input_mask[:,:1000],
                context_boundary=(10,200),
                labels= te1.labels.view(1, -1).to(device)
                )

In [11]:
output.encoder_last_hidden_state.shape

torch.Size([1, 450, 768])

In [29]:
torch.mean(torch.tensor([output['loss'],output['loss']],device = 'cuda',))

(tensor(2.9335, device='cuda:0'),
 tensor(2.9335, device='cuda:0', grad_fn=<NllLossBackward0>))

In [12]:
torch.all(output.logits==((output.logits+output.logits)/2))

tensor(True, device='cuda:0')

In [12]:
context_max_len_list

[250, 300, 450]

In [8]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=(390,700),
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=149,
        early_stopping=True,
        use_cache=True,
        num_beams=8,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['Former world champion Wendy Houvenaghel has accused British Cycling of sexism and discrimination.']

In [8]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

'A "medal at any cost" approach created a "culture of fear" at British Cycling, says former rider Wendy Houvenaghel.'

In [51]:
te1.input_ids.shape

torch.Size([1187])

In [58]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

Media playback is not supported on this device
The Olympic silver medallist accused the organisation of "ageism" and having "zero regard" for her welfare.
She is the latest high-profile cyclist to come forward after Jess Varnish, Nicole Cooke and Emma Pooley criticised the World Class programme.
Houvenaghel told the BBC she felt "vindicated" by a leaked draft report detailing British Cycling's failures.
The report said British Cycling "sanitised" its own investigation into claims former technical director Shane Sutton used sexist language towards Varnish, who went public last April about her treatment.
British Cycling subsequently admitted it did not pay "sufficient care and attention" to the wellbeing of staff and athletes at the expense of winning medals, an approach Houvenaghel attested to in her BBC interview.
Both Sutton and predecessor Sir Dave Brailsford have now left British Cycling.
Houvenaghel, 42, spoke to BBC Sport during its State of Sport week, which on Thursday examines 

In [22]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800

In [23]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
def generate():
    test_data_loader = DataLoader(test_dataset,batch_size=12,
                                sampler= SequentialSampler(test_dataset),
                                collate_fn= SmartCollator(
                pad_token_id=train_dataset.tokenizer.pad_token_id,
                max_len=configs.max_seq_len,
                context_max_len=context_max_len,
                context_sampling_bounds=context_sampling_bounds,
            
            )
                                )

    output_summaries =[]
    for batch in tqdm.tqdm(test_data_loader):
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        
        seq_len = b_input_ids.shape[1]

        context_boundary = compute_context_boundary(seq_len,
                                                    context_max_len=110)
        
        boundary_mask =  batch.get("boundary",)
        bb=generator.generate(input_ids=b_input_ids,
                attention_mask=b_input_mask,
                #context_boundary=boundary_mask,
                num_beams=10,
                do_sample=False,
                num_return_sequences=1,
                max_length=140)
        sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
        output_summaries+=sentences
    return output_summaries

In [25]:
base_output8 = generate()

100%|██████████| 945/945 [10:11<00:00,  1.54it/s]


In [10]:
#os.makedirs("outputs/multi_context/")

In [26]:
#context_enforcement/data/common.py
from context_enforcement.data.common import write_to_file
write_to_file(base_output8[:len(test_dataset)], "outputs/multi_context/best_base_new8")

Done


In [20]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

In [31]:
targets[690],base_output[690],base_output_8[690],base_output_45[690]#output_summaries2[690],output_summaries[690],output_summaries6[690],output_summaries8[690]

('Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.',
 "Uganda's Koffi Olomide has been taken into police custody in the capital, Kinshasa, after being arrested on suspicion of assault, the BBC understands.",
 "Uganda's Koffi Olomide has been taken into police custody in the capital, Kinshasa, after a video emerged showing him punching a woman in the face.",
 'Singer-songwriter Koffi Olomide has been arrested in the Democratic Republic of Congo on suspicion of assault after a video emerged showing him punching a woman in the face.')

In [13]:
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])


[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [27]:
scores = metrics.compute(predictions=base_output8,references=targets)
scores

{'bleu': 0.12420563480222273,
 'precisions': [0.4381499894596304,
  0.17908874941070982,
  0.09682579940300837,
  0.05619688787468894],
 'brevity_penalty': 0.864057575751863,
 'length_ratio': 0.8725121280166765,
 'translation_length': 227696,
 'reference_length': 260966,
 'meteor': 0.35420455825818364,
 'rouge1': 0.4026762188767841,
 'rouge2': 0.18481796867562256,
 'rougeL': 0.3305624212280963,
 'rougeLsum': 0.3305498661301927}

In [33]:
scores45 = metrics.compute(predictions=base_output_45,references=targets)
scores45

{'bleu': 0.12389472009506917,
 'precisions': [0.42998954172167264,
  0.17316442466234785,
  0.09333459931636916,
  0.05447402486628601],
 'brevity_penalty': 0.8882099297546506,
 'length_ratio': 0.894016845106259,
 'translation_length': 233308,
 'reference_length': 260966,
 'meteor': 0.3536737677235309,
 'rouge1': 0.4004153499833297,
 'rouge2': 0.1814263753204633,
 'rougeL': 0.32731796745661057,
 'rougeLsum': 0.3273520194107481}

In [36]:
scores8

{'bleu': 0.1242693218959997,
 'precisions': [0.4397112710255618,
  0.17984347696769157,
  0.09751670411397062,
  0.05682880711450953],
 'brevity_penalty': 0.8588878671603885,
 'length_ratio': 0.8679674746901895,
 'translation_length': 226510,
 'reference_length': 260966,
 'meteor': 0.3532861459937222,
 'rouge1': 0.4033676736623957,
 'rouge2': 0.18549005001850066,
 'rougeL': 0.3313132654462809,
 'rougeLsum': 0.3313118204210047}

In [17]:
scores

{'bleu': 0.12215108883184916,
 'precisions': [0.42916968255609556,
  0.17216560221044527,
  0.0922525258312802,
  0.0537097051014124],
 'brevity_penalty': 0.8830712157727976,
 'length_ratio': 0.8894032172773465,
 'translation_length': 232104,
 'reference_length': 260966,
 'meteor': 0.35137424074129014,
 'rouge1': 0.398632967632497,
 'rouge2': 0.1799538886903405,
 'rougeL': 0.32609557084990337,
 'rougeLsum': 0.32622241066447516}

In [24]:
scores4

{'bleu': 0.12300035067772054,
 'precisions': [0.4394282548476454,
  0.1790089177800281,
  0.09666579620313663,
  0.056324136455435936],
 'brevity_penalty': 0.8550152182505472,
 'length_ratio': 0.8645762283209307,
 'translation_length': 225625,
 'reference_length': 260966,
 'meteor': 0.35146299353791816,
 'rouge1': 0.4017681486403819,
 'rouge2': 0.18375560820687226,
 'rougeL': 0.32963068346151203,
 'rougeLsum': 0.3296792225053877}

In [18]:
scores5

{'bleu': 0.12202486703008819,
 'precisions': [0.43851006174759005,
  0.17777018935708405,
  0.09596328825046187,
  0.05604684262301944],
 'brevity_penalty': 0.8527557381282802,
 'length_ratio': 0.8626027911681982,
 'translation_length': 225110,
 'reference_length': 260966,
 'meteor': 0.3498951876777049,
 'rouge1': 0.40069971264338244,
 'rouge2': 0.18265325890345102,
 'rougeL': 0.3290151413411355,
 'rougeLsum': 0.3290728012119742}